## Load Packages

In [ ]:
using JuMP, Gurobi, DataFrames, CSV

## Loading Scenario Data, Day Ahead and Real Time price Scenarios

In [ ]:
#load summer_months
#summer_months = CSV.read("summer_months_new.csv", DataFrame);

In [ ]:
#pick 100 sample days from summer_months
#sample_days = sample(1:244, 100, replace=false);

In [ ]:
# filter the columns in summer_months from sample_days
#sample_days = summer_months[sample_days, :];

In [ ]:
#select!(sample_days, Not(:Date));

In [ ]:
#select!(sample_days, Not(:Index))

In [ ]:
#save sample_days to csv
#CSV.write("sample_days_new.csv", sample_days);

In [ ]:
sample_days = CSV.read("sample_days_new.csv", DataFrame);

In [ ]:
realtime = CSV.read("19_Jun_2019_realtime.csv", DataFrame);

In [ ]:
# read dayahead prices
dayahead = CSV.read("19_Jun_2019_dayahead.csv", DataFrame); # for 19th June 2019
#dayahead = CSV.read("30_July_2019_dayahead.csv", DataFrame); # for 30th July 2019

In [ ]:
dayaheadprices = dayahead.LZ_HOUSTON;

In [ ]:
samplematrix = Matrix(sample_days);

In [ ]:
# add dayaheadprices to samplematrix for each row
for i in 1:size(samplematrix, 1)
    samplematrix[i, :] = samplematrix[i,:] .+ dayaheadprices
end

In [ ]:
for i in 1:size(samplematrix, 1)
    for j in 1:size(samplematrix, 2)
        if samplematrix[i, j] < -251
            samplematrix[i, j] = samplematrix[i, j] * -1
        end
    end
end

In [ ]:
size(samplematrix)

In [ ]:
maximum(samplematrix)

In [ ]:
samplematrix = round.(samplematrix, digits=2);

## CVAR Battery Operation Model


In [ ]:
using JuMP, Gurobi, DataFrames

"""
optimize_battery_operations_cvar_ver3(prices::Matrix, battery_capacity::Float64, max_power::Float64,
charging_efficiency::Float64, discharging_efficiency::Float64, α::Float64, β::Float64)

Optimizes battery operations considering risk averseness using Conditional Value at Risk (CVaR).

# Arguments
- `prices::Matrix`: Matrix of electricity prices over time (num_scenarios x T)
- `battery_capacity::Float64`: Maximum capacity of the battery in kilowatt-hours (kWh)
- `max_power::Float64`: Maximum charging or discharging power of the battery in kilowatts (kW)
- `charging_efficiency::Float64`: Efficiency of the battery when charging (between 0 and 1)
- `discharging_efficiency::Float64`: Efficiency of the battery when discharging (between 0 and 1)
- `α::Float64`: Confidence level (between 0 and 1) for calculating CVaR
- `β::Float64`: Weighting factor for the risk term in the objective function

# Returns
- `results`: DataFrame containing optimization results
- `objective_values`: DataFrame containing objective function values (profits)
- `ζ_value`: Value of ζ, representing the CVaR
"""
function optimize_battery_operations_cvar_ver3(prices::Matrix, battery_capacity::Float64, max_power::Float64,
    charging_efficiency::Float64, discharging_efficiency::Float64, α::Float64, β::Float64)
    
    # Number of time periods (assuming 24-hour price scenarios)
    T = size(prices, 2)
    num_scenarios = size(prices, 1)

    # Create DataFrames to store the results
    results = DataFrame(scenario = Int[], hour = Int[], price = Float64[], soc = Float64[], charge = Float64[], discharge = Float64[])
    objective_values = DataFrame(scenario = Int[], profit = Float64[])

    # Initialize the model with the Gurobi solver
    model = Model(Gurobi.Optimizer)

    # Decision variables
    @variable(model, 0 <= soc[1:T] <= battery_capacity)  # State of charge
    @variable(model, 0 <= charge[1:T] <= max_power)  # Charge
    @variable(model, 0 <= discharge[1:T] <= max_power)  # Discharge
    @variable(model, ζ >= 0)  # CVaR auxiliary variable
    @variable(model, z[1:num_scenarios] >= 0)  # CVaR term
    @variable(model, u[1:T], Bin)  # Binary variable for controlling battery power

    # Objective function: Maximize profit
    @objective(model, Max, sum(0.01 * sum(prices[s, t] * (discharge[t] - charge[t]) for t in 1:T, s in 1:num_scenarios)) -
        β * (ζ + (1 / (1 - α)) * sum(0.01 * z[s] for s in 1:num_scenarios)))

    # Constraints
    @constraint(model, [s in 1:num_scenarios], soc[1] == 0)  # Initial SOC
    @constraint(model, [s in 1:num_scenarios], discharge[1] == 0)  # Prevent discharging at the first time step
    for t in 2:T
        for s in 1:num_scenarios
            # SOC dynamics
            @constraint(model, soc[t] == soc[t - 1] + charging_efficiency * charge[t - 1] -
                (1 / discharging_efficiency) * discharge[t - 1])
            
            # SOC should be greater than or equal to (discharge - charge)
            @constraint(model, soc[t] >= (discharge[t] - charge[t]))
            
            # CVaR constraints
            @constraint(model, z[s] >= sum(prices[s, t] * (discharge[t] - charge[t])) - ζ)
            
            # Power limits
            @constraint(model, discharge[t] - max_power * u[t] <= 0)
            @constraint(model, charge[t] - max_power * (1 - u[t]) <= 0)
        end
    end

    # Solve the optimization problem
    optimize!(model)
    
    # Save ζ value
    ζ_value = value(ζ)

    # Save the results
    for s in 1:num_scenarios
        for t in 1:T
            push!(results, (s, t, prices[s, t], value(soc[t]), value(charge[t]), value(discharge[t])))
        end
        push!(objective_values, (s, objective_value(model)))
    end
    
    return results, objective_values, ζ_value
end




In [ ]:
# Input parameters
battery_capacity = 100.0
max_power = 30.0
charging_efficiency = 0.95
discharging_efficiency = 0.95
α = 0.95
β = 0.2

In [ ]:
results_df, objective_values_df, zetaval = optimize_battery_operations_cvar_ver3(samplematrix, battery_capacity, max_power, charging_efficiency, discharging_efficiency, α, β)

In [ ]:
zetaval

In [ ]:
profit = sum(dayahead.LZ_HOUSTON.*(results_df[results_df.scenario .== 1, :].discharge.-results_df[results_df.scenario .== 1, :].charge))

In [ ]:
#save results_df to csv

beta = string(β)
CSV.write("Results/results_df_cvar_$(beta)_reformulated_ver3.csv", results_df)